In [2]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime 

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

#Yahoo!Finance 模組
import yfinance as yf 

import warnings
warnings.filterwarnings('ignore')

import nasdaqdatalink

In [110]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [111]:
print(tf.__version__)

2.12.0


In [112]:
tf.random.set_seed(42)
np.random.seed(42)

In [113]:
# https://finance.yahoo.com/most-active
# https://finance.yahoo.com/crypto/
symbol = "BTC-USD"
# Read data
df = yf.download(symbol,period='2y')
df_Indicators= yf.download(symbol,period='2y')
df.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-06-21,35641.144531,35721.640625,31295.935547,31676.693359,31676.693359,52809038594
2021-06-22,31622.376953,33292.453125,28893.621094,32505.660156,32505.660156,58964353058
2021-06-23,32515.714844,34753.410156,31772.632812,33723.027344,33723.027344,46317108925
2021-06-24,33682.800781,35228.851562,32385.214844,34662.437500,34662.437500,33123368116
2021-06-25,34659.105469,35487.246094,31350.884766,31637.779297,31637.779297,40230904226


In [114]:
df_Indicators.shape

(731, 6)

In [115]:
def SMA(close,n):   
    return close.rolling(window=n).mean()

def WMA(close, n):
    weights = np.array(range(1, n+1))
    sum_weights = np.sum(weights)

    res = close.rolling(window=n).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
    return res

def EMA(close, n):    
    res=close.ewm(span=n,min_periods=n).mean()
    return res

def RSI(close, n):
    # RSI
    change = close.diff(1)
    df["Gain"] = change.mask(change < 0, 0)
    df["Loss"] = abs(change.mask(change > 0, 0))
    df["AVG_Gain"] = df.Gain.rolling(n).mean()
    df["AVG_Loss"] = df.Loss.rolling(n).mean()
    df["RS"] = df["AVG_Gain"] / df["AVG_Loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))
    return df["RSI"]


def Stoch_RSI(close, n):
    #Stochastic RSI
    RSI=df["RSI"]
    LL_RSI = RSI.rolling(n).min()
    HH_RSI = RSI.rolling(n).max()         
    return (RSI - LL_RSI) / (HH_RSI - LL_RSI)

def BB(close, n):
    MA = pd.Series(close.rolling(n).mean())
    STD = pd.Series(close.rolling(n).std())
    bb1 = MA + 2 * STD
    df["Upper Bollinger Band"] = pd.Series(bb1)
    bb2 = MA - 2 * STD
    df["Lower Bollinger Band"] = pd.Series(bb2)
    return df["Upper Bollinger Band"],df["Lower Bollinger Band"]

def MACD(close):
    df['EMA12'] = close.ewm(span=12, adjust=False).mean() 
    df['EMA26'] = close.ewm(span=26, adjust= False).mean() 
    return df['EMA12'] - df['EMA26'] 


# 指標副程式(資料,天數)
df_Indicators['SMA_7']=SMA(df['Adj Close'],7)
df_Indicators['SMA_14']=SMA(df['Adj Close'],14)

df_Indicators['EMA_7']=EMA(df['Adj Close'],7)
df_Indicators['EMA_14']=EMA(df['Adj Close'],14)

df_Indicators['RSI']=RSI(df['Adj Close'],14)


df_Indicators['Stoch_RSI']=Stoch_RSI(df['Adj Close'],14)

df_Indicators['Upper_BB']=BB(df['Adj Close'],20)[0]
df_Indicators['Lower_BB']=BB(df['Adj Close'],20)[1]

df_Indicators['MACD']=MACD(df['Adj Close'])
df_Indicators= df_Indicators.dropna()
df_Indicators.head()

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
Date,,,,,,,,,,,,,,,
2021-07-18,31533.884766,32398.996094,31215.492188,31796.810547,31796.810547,18787986667,32173.195592,32963.213309,32166.075902,32700.499547,30.031418,0.000000,35992.131690,30989.777489,-317.401505
2021-07-19,31800.011719,31885.859375,30563.734375,30817.832031,30817.832031,20434789545,31839.193359,32754.058175,31828.934658,32445.456300,32.098048,0.073054,35748.926483,30727.988165,-434.231771
2021-07-20,30838.285156,31006.187500,29360.955078,29807.347656,29807.347656,23148267245,31425.667969,32437.783343,31323.447637,32088.835787,24.551583,0.000000,35771.180993,30182.384828,-601.425386
2021-07-21,29796.285156,32752.326172,29526.183594,32110.693359,32110.693359,28203024559,31324.003069,32313.166574,31520.285431,32091.785055,41.788509,0.510438,35708.971376,30098.452062,-541.821006
2021-07-22,32138.873047,32576.400391,31745.298828,32313.105469,32313.105469,19555230518,31400.056641,32272.861886,31718.510353,32121.600433,47.135005,0.668763,35600.983279,30048.046018,-472.800985


In [116]:
df_Indicators.shape

(704, 15)

In [117]:
n = len(df)
train_df = df_Indicators[0:int(n*0.7)]
val_df = df_Indicators[int(n*0.7):int(n*0.9)]
test_df = df_Indicators[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((511, 15), (146, 15), (47, 15))

In [118]:
#將值縮限在0~1之間，加速訓練時間
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [119]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')

In [6]:
# Bitcoin Average Block Size
# mydata = nasdaqdatalink.get("BCHAIN/AVBLS")

# Bitcoin Hash Rate
# mydata = nasdaqdatalink.get("BCHAIN/HRATE")

# Bitcoin api.blockchain Size
# mydata = nasdaqdatalink.get("BCHAIN / BLCHS")


mydata

SyntaxError: Invalid format used for Nasdaq Data Link database code.         The correct format is: `DATABASE_CODE/DATASET_CODE` (<string>)